In [1]:
!pip install transformers peft accelerate bitsandbytes

In [2]:
import os
import sys

import torch
import transformers
import accelerate
import bitsandbytes
from peft import PeftModel
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer, BitsAndBytesConfig


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda112.so
CUDA SETUP: CUDA runtime path found: /opt/conda/envs/Python-3.9-CUDA/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 112
CUDA SETUP: Loading binary /opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda112.so...


/opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/opt/conda/envs/Python-3.9-CUDA/lib/libcudart.so.11.0'), PosixPath('/opt/conda/envs/Python-3.9-CUDA/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


In [3]:
import json
import os.path as osp
from typing import Union


class Prompter(object):
    __slots__ = ("template", "_verbose")

    def __init__(self, template_name: str = "", verbose: bool = False):
        self._verbose = verbose
        if not template_name:
            # Enforce the default here, so the constructor can be called with '' and will not break.
            template_name = "alpaca"
        file_name = osp.join("./templates", f"{template_name}.json")
        if not osp.exists(file_name):
            raise ValueError(f"Can't read {file_name}")
        with open(file_name) as fp:
            self.template = json.load(fp)
        if self._verbose:
            print(
                f"Using prompt template {template_name}: {self.template['description']}"
            )

    def generate_prompt(
        self,
        instruction: str,
        input: Union[None, str] = None,
        label: Union[None, str] = None,
    ) -> str:
        # returns the full prompt from instruction and optional input
        # if a label (=response, =output) is provided, it's also appended.
        if input:
            res = self.template["prompt_input"].format(
                instruction=instruction, input=input
            )
        else:
            res = self.template["prompt_no_input"].format(
                instruction=instruction
            )
        if label:
            res = f"{res}{label}"
        if self._verbose:
            print(res)
        return res

    def get_response(self, output: str) -> str:
        return output.split(self.template["response_split"])[1].strip()

In [6]:
cache_directory = "/mnts/llm3"

# params
# base_model = "huggyllama/llama-65b"
base_model = "huggyllama/llama-30b"
device = "cuda"
# lora_weights = "chansung/alpaca-lora-65b"
lora_weights = "baseten/alpaca-30b"

prompter = Prompter("")

tokenizer = LlamaTokenizer.from_pretrained(base_model)

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [7]:
import accelerate
import bitsandbytes

load_8bit = True
# to run this with load_8bit=True, you need to run the cp commands from the below two issues:
# https://github.com/tloen/alpaca-lora/issues/46
# https://github.com/tloen/alpaca-lora/issues/294
model = LlamaForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=load_8bit,
    torch_dtype=torch.float16,
    device_map="auto",
    cache_dir=cache_directory,
)
model = PeftModel.from_pretrained(
    model,
    lora_weights,
    torch_dtype=torch.float16,
    cache_dir=cache_directory,
)

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

RuntimeError: unable to mmap 9818304536 bytes from file </mnts/llm3/models--huggyllama--llama-30b/snapshots/2b1edcdb3c7ced7bce6c1aa75c94545777c3118b/model-00001-of-00007.safetensors>: Cannot allocate memory (12)

In [34]:
model

NameError: name 'model' is not defined